In [15]:
import json
import os
import glob
import urllib
import functools
import numpy as np
import pandas as pd

In [2]:
# Downloads contract json files and loads it into DATA

if not os.path.exists("data"):
    os.makedirs("data")
    
for url in (
    "https://techinquiry.org/SiliconValley-Military/subcontracts/microsoft_cloud.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/microsoft_noncloud.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/amazon.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/google.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/facebook.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/nvidia.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/twitter.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/palantir.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/ida.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/mitre.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/johns_hopkins.json"
):
    
    path = os.path.join('./data/', url.split('/')[-1])

    if not os.path.isfile(path):
        urllib.request.urlretrieve(url, path)

DATA = {}

for path in glob.glob('./data/*.json'):
    company_name = os.path.splitext(os.path.basename(path))[0]
    
    with open(path, 'r') as f:
        DATA[company_name] = json.load(f)
    


In [3]:
# flatten data in a map of company name to list of contracts

CONTRACTS = {}

for company in DATA.keys():
    company_contracts = []
    
    for intermediary in DATA[company]:
        for agency in DATA[company][intermediary]:
            for contract in DATA[company][intermediary][agency]:
                company_contracts.append(contract | { "intermediary": intermediary, "agency":  agency })
    
    CONTRACTS[company] = company_contracts

    


In [16]:
# Summary of base_and_all_options_value

stats = {}

sumkey = "base_and_all_options_value"

for company, contracts in CONTRACTS.items():
        amount = sum(map(lambda x: float(x[sumkey]), contracts))
        stats[company] =  '${:0,.2f}'.format(amount)


stats

{'nvidia': '$31,742,561.60',
 'microsoft_noncloud': '$5,647,780,024.40',
 'google': '$68,046,317.56',
 'palantir': '$11,798,965.21',
 'amazon': '$291,237,334.80',
 'twitter': '$5,710,057.53',
 'johns_hopkins': '$55,715.00',
 'microsoft_cloud': '$19,904,206,668.23',
 'mitre': '$2,967,641.04',
 'facebook': '$6,833,447.03',
 'ida': '$273,804.89'}